In [8]:
import random
import pandas as pd
from sklearn.metrics import classification_report, roc_auc_score
import csv
import numpy as np
from tabulate import tabulate
from classification import classification
from rule_gen import rule_generation
from daehan_mlutil import utilities
from util import util

with open('../data/dataset.csv', 'r') as file:
    dataset = [list(filter(None, row)) for row in csv.reader(file)]

min_support = 0.06
transactions_df = util.convert_trans_to_df(dataset)
transactions_df

,0,1,ACETAMINOPHEN,ALBUMIN HUMAN 25%,ALBUMIN HUMAN 5%,ALBUTEROL 0.083%,ALBUTEROL 90 MCG,ALBUTEROL CONCENTRATED 0.5%,ALLOPURINOL,ALPRAZOLAM,...,TACROLIMUS,TAMSULOSIN,THIAMINE,TRAMADOL,TRAZODONE,VANCOMYCIN,VANCOMYCIN 1 G/200 ML D5W,VANCOMYCIN 1.25 G/250 ML NS,VANCOMYCIN 1.5 G/500 ML NS,WARFARIN
0,True,False,True,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
1,True,False,True,False,False,False,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4610,True,False,True,False,False,False,True,True,False,False,...,False,False,False,True,True,False,True,True,True,False
4611,True,False,False,False,False,False,True,True,False,False,...,False,False,False,False,False,True,False,False,False,False
4612,True,False,False,False,True,False,False,False,False,False,...,False,False,True,False,False,True,False,False,False,False
4613,False,True,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [5]:
random.seed(0)
transactions_0 = pd.DataFrame(
    transactions_df[transactions_df['0']].reset_index(drop=True))
transactions_1 = pd.DataFrame(
    transactions_df[transactions_df['1']].reset_index(drop=True))

In [7]:
indices = list(range(0, len(transactions_0)))
random.shuffle(indices)
test_set_0 = transactions_0.iloc[indices[:417], :].reset_index(drop=True)
training_set_0 = transactions_0.iloc[indices[417:], :].reset_index(drop=True)

indices = list(range(0, len(transactions_1)))
random.shuffle(indices)
test_set_1 = transactions_1.iloc[indices[:43], :].reset_index(drop=True)
training_set_1 = transactions_1.iloc[indices[43:], :].reset_index(drop=True)

training_set = pd.concat([training_set_0, training_set_1])
test_set = pd.concat([test_set_0, test_set_1])
training_set

,0,1,ACETAMINOPHEN,ALBUMIN HUMAN 25%,ALBUMIN HUMAN 5%,ALBUTEROL 0.083%,ALBUTEROL 90 MCG,ALBUTEROL CONCENTRATED 0.5%,ALLOPURINOL,ALPRAZOLAM,...,TACROLIMUS,TAMSULOSIN,THIAMINE,TRAMADOL,TRAZODONE,VANCOMYCIN,VANCOMYCIN 1 G/200 ML D5W,VANCOMYCIN 1.25 G/250 ML NS,VANCOMYCIN 1.5 G/500 ML NS,WARFARIN
0,True,False,True,False,False,False,True,True,False,False,...,False,False,False,True,True,True,True,False,False,True
1,True,False,True,True,True,False,False,False,False,False,...,False,False,False,True,False,True,True,False,False,False
2,True,False,True,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
3,True,False,True,True,True,False,True,False,False,False,...,True,False,False,False,False,True,True,True,True,True
4,True,False,False,False,False,False,True,True,False,False,...,False,False,False,False,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
389,False,True,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
390,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
391,False,True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,False


In [9]:
test_set

,0,1,ACETAMINOPHEN,ALBUMIN HUMAN 25%,ALBUMIN HUMAN 5%,ALBUTEROL 0.083%,ALBUTEROL 90 MCG,ALBUTEROL CONCENTRATED 0.5%,ALLOPURINOL,ALPRAZOLAM,...,TACROLIMUS,TAMSULOSIN,THIAMINE,TRAMADOL,TRAZODONE,VANCOMYCIN,VANCOMYCIN 1 G/200 ML D5W,VANCOMYCIN 1.25 G/250 ML NS,VANCOMYCIN 1.5 G/500 ML NS,WARFARIN
0,True,False,True,False,False,False,True,True,False,False,...,False,False,False,False,False,True,True,False,True,False
1,True,False,True,False,False,False,True,True,False,False,...,False,False,False,True,True,False,False,False,False,False
2,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
4,True,False,True,False,False,False,True,True,False,True,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,False,True,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
39,False,True,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
40,False,True,True,False,True,False,False,False,False,False,...,False,False,False,False,True,False,True,True,False,False
41,False,True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [13]:
import multiprocessing
import pandas as pd
from posneg_rule_gen.posneg_rule_generation import ponerg
from rule_gen import apriori_mlx
from util import util
from daehan_mlutil import utilities

def ponerg_parallel(item):
    return ponerg(item, classes, class_support_count_dict, transactions_df)

rules = []
transactions_df = training_set
X_df = pd.DataFrame(transactions_df.drop(['1', '0'], axis=1))
classes = [frozenset(['0']), frozenset(['1'])]
class_support_count_dict = util.get_support_count_dict_df(classes, transactions_df)

f1, previous_itemset_arr = apriori_mlx.apriori_of_size_1(X_df, min_support=min_support)
f1 = f1.tolist()
freq_itemsets = [f1]
for item in f1:
    rules.extend(ponerg(item, classes, class_support_count_dict, transactions_df))
np.expand_dims(np.array(np.array(f1)), axis=1)

array([[frozenset({'ACETAMINOPHEN'})],
       [frozenset({'ALBUMIN HUMAN 25%'})],
       [frozenset({'ALBUMIN HUMAN 5%'})],
       [frozenset({'ALBUTEROL 90 MCG'})],
       [frozenset({'ALBUTEROL CONCENTRATED 0.5%'})],
       [frozenset({'AMINOCAPROIC ACID'})],
       [frozenset({'AMIODARONE'})],
       [frozenset({'AMLODIPINE'})],
       [frozenset({'ASPIRIN'})],
       [frozenset({'ASPIRIN EC'})],
       [frozenset({"ASPIRIN, CHILDREN'S"})],
       [frozenset({'ATORVASTATIN'})],
       [frozenset({'BACITRACIN'})],
       [frozenset({'BISACODYL'})],
       [frozenset({'CALCIUM CARBONATE'})],
       [frozenset({'CALCIUM GLUCONATE 2 G/50 ML NS'})],
       [frozenset({'CEFAZOLIN'})],
       [frozenset({'CEFAZOLIN 2 G/100 ML NS'})],
       [frozenset({'CEFEPIME'})],
       [frozenset({'CEFTRIAXONE'})],
       [frozenset({'CHLORHEXIDINE GLUCONATE 0.12%'})],
       [frozenset({'CLOPIDOGREL'})],
       [frozenset({'DEXAMETHASONE'})],
       [frozenset({'DEXTROSE 5%-NACL 0.9%'})],
       [fro

In [14]:
import time

start_time = time.time()
k = 0
while freq_itemsets[k] is not None and len(freq_itemsets[k]) > 0:
    k_freq_itemsets, previous_itemset_arr = apriori_mlx.apriori_of_size_k(
        X_df, previous_itemset_arr, min_support=min_support, k=k + 2, low_memory=True)
    if not k_freq_itemsets.empty:
        k_freq_itemsets = k_freq_itemsets.tolist()
        # for item in k_freq_itemsets_x_c:
        #     rules.extend(ponerg(item, c, len(transactions_per_c), transactions_df))
        with multiprocessing.Pool() as pool:
            result = pool.map(ponerg_parallel, k_freq_itemsets)
        rules_to_extend = [x[0] for x in result if x != []]
        rules.extend(rules_to_extend)
        freq_itemsets.append(k_freq_itemsets)
    else:
        freq_itemsets.append(None)
    k += 1
time_sec = time.time() - start_time
time_min = time_sec / 60
print("\nProcessing time of %s(): %.2f seconds (%.2f minutes)."
      % ("rule_gen", time.time() - start_time, time_min))
np.expand_dims(np.array(np.array(rules)), axis=1)


Processing time of whole code(): 21.08 seconds (0.35 minutes).


array([[{'antecedent': frozenset({'ACETAMINOPHEN'}), 'consequent': '0', 'confidence': 1.0280239160480256}],
       [{'antecedent': frozenset({'ALBUMIN HUMAN 25%'}), 'consequent': '1', 'confidence': 1.3611326964391828}],
       [{'antecedent': frozenset({'ALBUMIN HUMAN 5%'}), 'consequent': '0', 'confidence': 1.03012667157404}],
       ...,
       [{'antecedent': frozenset({'OXYCODONE IR', 'MAGNESIUM SULF 2 G/50 ML NS', 'FUROSEMIDE', 'METOPROLOL TARTRATE', 'ACETAMINOPHEN', 'DEXTROSE 50%-WATER', 'HEPARIN 5,000 UNIT/ML', 'DOCUSATE SODIUM', 'ONDANSETRON', 'FENTANYL CITRATE (PF)', 'POTASSIUM CHLORIDE ER'}), 'consequent': '0', 'confidence': 1.0962234283129806}],
       [{'antecedent': frozenset({'OXYCODONE IR', 'MAGNESIUM SULF 2 G/50 ML NS', 'FUROSEMIDE', 'ACETAMINOPHEN', 'DEXTROSE 50%-WATER', 'HEPARIN 5,000 UNIT/ML', 'DOCUSATE SODIUM', 'VANCOMYCIN', 'ONDANSETRON', 'FENTANYL CITRATE (PF)', 'POTASSIUM CHLORIDE ER'}), 'consequent': '0', 'confidence': 1.0956299840510368}],
       [{'antecedent':

In [25]:
rules_0 = [rule for rule in rules if rule['consequent'] == '0']
rules_1 = [rule for rule in rules if rule['consequent'] == '1']
sorted_rules = sorted(rules, key=lambda d: d['confidence'], reverse=True)

training_transactions_1 = training_set[training_set['1']].drop(['1', '0'], axis=1).apply(
    lambda row: frozenset(row.index[row]), axis=1).tolist()
scores_training = [classification.classify(object_o, sorted_rules, 0.1) for object_o in training_transactions_1]
np.array(scores_training)

array([[1.        , 6.10763055],
       [1.        , 5.1670206 ],
       [1.        , 3.24699088],
       [1.        , 2.691896  ],
       [1.        , 4.26967117],
       [1.        , 4.43631193],
       [1.        , 6.10763055],
       [1.        , 4.19647681],
       [1.        , 6.10763055],
       [1.        , 4.5841782 ],
       [1.        , 5.1670206 ],
       [1.        , 4.11743834],
       [1.        , 4.43631193],
       [1.        , 6.10763055],
       [1.        , 4.43631193],
       [1.        , 5.1670206 ],
       [1.        , 6.10763055],
       [1.        , 4.19647681],
       [1.        , 4.27769687],
       [1.        , 4.37236382],
       [1.        , 4.19647681],
       [1.        , 3.14885496],
       [1.        , 6.10763055],
       [1.        , 5.1670206 ],
       [1.        , 5.1670206 ],
       [1.        , 4.5841782 ],
       [1.        , 5.1670206 ],
       [1.        , 6.10763055],
       [1.        , 6.10763055],
       [1.        , 6.10763055],
       [1.

In [22]:
mean = np.mean(scores_training)
mean

2.9934914259950047

In [19]:
test_transactions = test_set.drop(['1', '0'], axis=1).apply(lambda row: frozenset(row.index[row]), axis=1).tolist()
scores_test = [classification.predict_proba(object_o, sorted_rules) for object_o in test_transactions]
scores_test = np.array(scores_test)
scores_test

array([3.02955225, 1.87512603, 1.95442694, 2.19139486, 4.37236382,
       3.39452118, 1.92877037, 2.90949169, 1.94052565, 2.12971607,
       3.36176874, 6.10763055, 1.74392067, 2.69578176, 4.19647681,
       3.06267418, 1.25436667, 3.06267418, 2.32363057, 2.77662117,
       3.99095091, 3.784715  , 4.5841782 , 1.58192305, 2.51147963,
       2.76565897, 2.84644745, 2.20022694, 3.77877108, 2.22176126,
       3.39830971, 2.45282443, 1.90987442, 2.2655398 , 3.51024349,
       2.90949169, 2.16445273, 3.36214208, 3.62016218, 3.47204029,
       1.84815244, 3.07901433, 2.21061763, 1.09099662, 2.51147963,
       1.83594643, 3.0341253 , 3.24699088, 1.84950073, 3.24699088,
       3.72671171, 1.98339786, 3.38528936, 2.2780566 , 3.39452118,
       6.10763055, 2.78422211, 2.34944869, 4.43631193, 1.6265414 ,
       3.63560933, 2.51147963, 3.71248762, 3.50857934, 5.1670206 ,
       3.39452118, 3.55805931, 2.13564885, 2.21061763, 4.11743834,
       3.52417303, 3.62016218, 4.34601575, 3.67101357, 1.61630

In [20]:
y_test = test_set.apply(lambda row: 0 if row['0'] else 1, axis=1).tolist()
y_test = np.array(y_test)
y_test

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [18]:
y_pred = np.zeros(len(scores_test), dtype=int)
y_pred[scores_test >= mean] = 1
y_pred

array([1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,